# AI based Natural disaster analysis

In [1]:
pwd


'/home/wsuser/work'

### Importing Neccessary Libraries

In [2]:
!pip install keras==2.2.4
!pip install tensorflow==2.2.0

In [4]:
import numpy as np#used for numerical analysis

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_8afb15d112f14e799e8fde1f1bb9dacd = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_8afb15d112f14e799e8fde1f1bb9dacd = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_8afb15d112f14e799e8fde1f1bb9dacd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='diRTl2M3775Lk6XgwYxC1LZ5TWBvpeTTaRRT04VunvSC',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_8afb15d112f14e799e8fde1f1bb9dacd)


streaming_body_2 = client_8afb15d112f14e799e8fde1f1bb9dacd.get_object(Bucket='naturaldisasters-donotdelete-pr-rldfhdatayr1mr', Key='dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [5]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [6]:
tensorflow.__version__

'2.2.0'

In [7]:
tensorflow.keras.__version__

'2.3.0-tf'

In [8]:
pwd


'/home/wsuser/work'

In [9]:
import os
filename = os.listdir('/home/wsuser/work/dataset/test_set')

### Image Data Agumentation

In [10]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [11]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory('/home/wsuser/work/dataset/train_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory('/home/wsuser/work/dataset/test_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical') 

Found 742 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [12]:
print(x_train.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [13]:
print(x_test.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [14]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 220, 1: 156, 2: 198, 3: 168})

### Creating the model

In [15]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and poolingo
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2



2021-11-28 10:56:45.540357: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow
2021-11-28 10:56:45.540410: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: UNKNOWN ERROR (303)


In [16]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 3872)              0

### Compiling the model

In [17]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [18]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=40, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
149/149 [==============================] - 34s 228ms/step - loss: 1.2206 - accuracy: 0.4367 - val_loss: 1.1049 - val_accuracy: 0.5505
Epoch 2/40
149/149 [==============================] - 33s 221ms/step - loss: 1.0992 - accuracy: 0.5054 - val_loss: 1.0189 - val_accuracy: 0.5657
Epoch 3/40
149/149 [==============================] - 32s 218ms/step - loss: 1.0572 - accuracy: 0.5526 - val_loss: 1.0673 - val_accuracy: 0.5556
Epoch 4/40
149/149 [==============================] - 33s 219ms/step - loss: 0.9359 - accuracy: 0.5876 - val_loss: 0.9340 - val_accuracy: 0.6212
Epoch 5/40
149/149 [==============================] - 33s 219ms/step - loss: 0.8087 - accuracy: 0.6307 - val_loss: 0.9783 - val_accuracy: 0.6364
Epoch 6/40
149/149 [==============================] - 32s 218ms/step - loss: 0.7887 - accuracy: 0.6765 - val_loss: 0.9030 - val_accuracy: 0.6212
Epoch 7/40
149/149 [==============================] - 

### Saving our model

In [38]:
# Save the model
classifier.save('disaster_f.h5')

In [37]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

In [43]:
ls -l


total 11816
drwxrwx--- 4 wsuser wscommon    4096 Nov 28 09:28 dataset/
-rw-rw---- 1 wsuser wscommon 6358984 Nov 28 11:50 disaster_f.h5
-rw-rw---- 1 wsuser wscommon 5727832 Nov 28 12:00 image-classification-model_new.tgz
-rw-rw---- 1 wsuser wscommon    3854 Nov 28 11:50 model-bw.json


In [41]:
!tar -zcvf image-classification-model_new.tgz disaster_f.h5

disaster_f.h5


In [42]:
!pip install ibm_watson_machine_learning --upgrade


     |████████████████████████████████| 1.8 MB 29.4 MB/s eta 0:00:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.173
    Uninstalling ibm-watson-machine-learning-1.0.173:
      Successfully uninstalled ibm-watson-machine-learning-1.0.173


In [47]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"KDjFaE3CaF3dFaKfFmlfkhI5rG5TAM6fWIqZOBKIWy92"
                  }
client = APIClient(wml_credentials)

In [48]:
client = APIClient(wml_credentials)

In [49]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [50]:
space_uid = guid_from_space_name(client, 'imageclassification')
print(space_uid)

e11e4fae-ea39-4045-a975-1bbd7fb6758b


In [51]:
client.set.default_space(space_uid)

'SUCCESS'

In [52]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [53]:
software_spec_uid = client.software_specifications.get_uid_by_name("default_py3.8")
software_spec_uid

'ab9e1b80-f2ce-592c-a7d2-4f2344f77194'

In [59]:
 model_details = client.repository.store_model(model="image-classification-model_new.tgz",meta_props={
 client.repository.ModelMetaNames.NAME:"CNN",
 client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
 client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid }
                                             )
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [72]:
model_id

'da90b78d-7a6b-4fd6-bf9e-3d52f97cfa47'

### Predicting our results

In [44]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
#model = load_model("disaster_f.h5") #loading the model for testing

In [45]:
ls

dataset/  disaster_f.h5  image-classification-model_new.tgz  model-bw.json


In [46]:
img = image.load_img("/home/wsuser/work/dataset/test_set/Cyclone/867.jpeg",grayscale=False,
                     target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = classifier.predict_classes(x)#predicting the classes
pred

FileNotFoundError: [Errno 2] No such file or directory: '/home/wsuser/work/dataset/test_set/Cyclone/867.jpeg'

In [ ]:
index=['Cyclone','Earthquake','Flood','Wildfire']
result=str(index[pred[0]])
result

In [ ]:
!pip install jupyterthemes as jt

In [ ]:
!jt -t monokai